# LLM Fine-Tuning with QLoRA (Qwen2.5-3B)

## Environment & Version Checks

In [1]:
import transformers
print(transformers.__version__)

import sys
sys.path.append("..")


5.0.0


## Dependency Installation (Jupyter-safe)

In [2]:
#%pip install peft==0.10.0
#%pip install -U "bitsandbytes>=0.46.1"
import json

## Global Configuration

In [3]:
OUTPUT_FOLDER = "../outputs"
DATA_FOLDER = "../data"
MODEL_NAME = "Qwen/Qwen2.5-3B"
LORA_PATH = OUTPUT_FOLDER + "/checkpoints/lora_qwen/"
VAL_JSON = DATA_FOLDER + "/processed/val.json"
TEST_JSON = DATA_FOLDER + "/processed/test.json"
LABELS = ["negative", "neutral", "positive"]

## Trainer Initialization (QLoRA Setup)

In [4]:
from src.DecoderTrainer import DecoderTrainer
from transformers import TrainingArguments

trainer = DecoderTrainer(
    model_name=MODEL_NAME,
    load_in_4bit=True
)

trainer.configure_lora(r=8, lora_alpha=32)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.059693335438076124


## Training Configuration

In [5]:
training_args = TrainingArguments(
    # ===== keep ALL your existing arguments here =====
    output_dir="outputs/logs",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    fp16=True,
    fp16_full_eval=False,
    num_train_epochs=1,
    eval_strategy="epoch",
    logging_steps=50,
    save_strategy="epoch",
    report_to="none",

    # ===== ADD ONLY THESE =====
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    save_total_limit=1,
)


## Sanity Check: Model Generation (Before Training)

In [6]:
prompt = "Classify the sentiment: Copper prices fell due to weak demand."
print(trainer.generate(prompt=prompt, max_new_tokens=50))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Classify the sentiment: Copper prices fell due to weak demand. The sentiment of the statement is negative.


## Dataset Loading

In [7]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={
        "train": "../data/processed/train.json",
        "validation": "../data/processed/val.json",
        "test": "../data/processed/test.json",
    }
)

dataset["train"] = dataset["train"].select([0])
dataset["validation"] = dataset["validation"].select([0])
dataset["test"] = dataset["test"].select([0])

## Prompt Formatting (Instruction-Tuning Style)

In [8]:
tokenizer = trainer.tokenizer

def format_prompt(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output = example["output"]

    text = (
        "### Instruction:\n"
        f"{instruction}\n\n"
        "### Input:\n"
        f"{input_text}\n\n"
        "### Response:\n"
        f"{output}{tokenizer.eos_token}"
    )

    return {"text": text}

dataset = dataset.map(
    format_prompt,
    batched=False,
    num_proc=1,
    desc="Formatting prompts"
)

## Tokenization & Data Collation

In [9]:
from transformers import DataCollatorForLanguageModeling

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=128,
        padding=False,   # dynamic padding later
    )

tokenized_ds = dataset.map(
    tokenize,
    batched=True,
    batch_size=128,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,   # causal LM
)

## Sequence Length Analysis (Sanity Check)

In [10]:
def add_token_length(batch):
    return {
        "token_len": [len(ids) for ids in batch["input_ids"]]
    }

tokenized_ds = tokenized_ds.map(
    add_token_length,
    batched=True,
)

import numpy as np

lengths = tokenized_ds["train"]["token_len"]

print({
    "min": min(lengths),
    "max": max(lengths),
    "mean": np.mean(lengths),
    "p95": np.percentile(lengths, 95),
})


{'min': 63, 'max': 63, 'mean': np.float64(63.0), 'p95': np.float64(63.0)}


## Model Training

In [11]:
trainer.train(
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    training_args=training_args,
    data_collator=data_collator,
    classification_eval_fn=lambda: trainer.evaluate_classification(
        test_path=VAL_JSON,
        labels=LABELS,
    ),
)


Epoch,Training Loss,Validation Loss
1,No log,2.519675


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for


[Classification Metrics]
accuracy: 0.5579
precision: 0.1860
recall: 0.3333
f1: 0.2387
auc_ovr: 0.8486
classification_report:               precision    recall  f1-score   support

    negative       0.00      0.00      0.00        37
     neutral       0.56      1.00      0.72       135
    positive       0.00      0.00      0.00        70

    accuracy                           0.56       242
   macro avg       0.19      0.33      0.24       242
weighted avg       0.31      0.56      0.40       242



## Save LoRA Adapters

In [12]:
#trainer.save_lora_adapters(OUTPUT_FOLDER + "/checkpoints/lora_qwen")

In [12]:
logs = trainer.trainer.state.best_metric
print(logs)

0.23872679045092837


## Verify Trainable Parameters (LoRA Only)

In [ ]:
trainable = [
    (n, p.requires_grad)
    for n, p in trainer.model.named_parameters()
    if p.requires_grad
]

print(f"Trainable params: {len(trainable)}")
trainable[:10]
